In [ ]:
pip install imbalanced-learn

In [2]:
pip install pytorch-tabnet

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   ---------------------------------------- 44.5/44.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.2/159.7 MB 4.6 MB/s eta 0:00:35
   ---------------------------------------- 0.5/159.7 MB 5.9 MB/s eta 0:00:27
   ---------------------------------------- 0.7/159.7 MB 5.8 MB/s eta 0:00:28
   ---------------------------------------- 1.1/159.7 MB 5.7 MB/s eta 0:00:28
   ---------------------------------------- 1.4/159.7 MB 6.2 MB/s eta 0:00:26
   ---------------------------------------- 1.6/159.7 MB 6.2 MB/s eta 0:00:26
   ---------------------------------------- 1.9/159.7 MB 6.0 MB/s eta 0:00:27
    --------------------------------------- 2.2/159.7 MB 6.0 MB/s eta 0:00:27
    --------------------------------------- 2.6/159.7 MB 6.3 MB/s eta 0:00:25
    ----

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.re

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score, precision_score,roc_auc_score,f1_score,confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [18]:
# df = pd.read_csv("윈저라이징_inf제거.csv",index_col = 0)
# df = df.drop(columns = ['코스닥연수익률 (%)', '코스피연수익률(%)','코스닥변동성', '코스피변동성'])
# X_data = df.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
# X_data = df.loc[:,['순이익증가율',
#                     '종업원수증가율',
#                     '총자본투자효율',
#                     '부가가치율',
#                     '매출액순이익률',
#                     '자기자본정상영업이익률',
#                     'CASH FLOW 대 매출액비율',
#                     'CASH FLOW 대 차입금비율']]
# y_data = df['부실']

In [ ]:
all_features = pd.read_csv('C:/Users/SAMSUNG/Documents/vscode/2024/ML/프로젝트2/data/윈저라이징_inf제거.csv',index_col=0)
all_features = all_features.drop(columns = ['코스닥연수익률 (%)', '코스피연수익률(%)','코스닥변동성', '코스피변동성'])
X_data = all_features.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
y_data = all_features['부실']

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.3,stratify=y_data,random_state = 42)

In [20]:
#SMOTE-Tomek
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_train_resampled_t, y_train_resampled_t = smoteto.fit_resample(X_train, y_train)

In [104]:
#Borderline SMOTE
from imblearn.over_sampling import BorderlineSMOTE
X_train_resampled_b, y_train_resampled_b = BorderlineSMOTE(random_state=0).fit_resample(X_train, y_train)

In [116]:
# ADASYN
from imblearn.over_sampling import ADASYN
X_train_resampled_a, y_train_resampled_a = ADASYN(random_state=42).fit_resample(X_train, y_train)

In [21]:
X_train, y_train = X_train_resampled_t, y_train_resampled_t

In [22]:
df = pd.DataFrame()

In [23]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred_train = logit.predict(X_train)
y_pred_test = logit.predict(X_test)

log_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
log_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['logit_train'] = log_train
df['logit_test'] = log_test

print(confusion_matrix(y_test,y_pred_test))

[[3095  297]
 [  43  262]]


In [24]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

rf_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
rf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['rf_train'] = rf_train
df['rf_test'] = rf_test


print(confusion_matrix(y_test,y_pred_test))

[[3174  218]
 [  43  262]]


In [25]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred_train = xgb.predict(X_train)
y_pred_test = xgb.predict(X_test)

xgb_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
xgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['xgb_train'] = xgb_train
df['xgb_test'] = xgb_test


print(confusion_matrix(y_test,y_pred_test))

[[3169  223]
 [  47  258]]


In [26]:
lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)
y_pred_train = lgb.predict(X_train)
y_pred_test = lgb.predict(X_test)

lgb_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
lgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['lgb_train'] = lgb_train
df['lgb_test'] = lgb_test


print(confusion_matrix(y_test,y_pred_test))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7913, number of negative: 7877
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000525 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 15790, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501140 -> initscore=0.004560
[LightGBM] [Info] Start training from score 0.004560
[[3147  245]
 [  41  264]]


In [27]:
cat = CatBoostClassifier(random_state=42, verbose=0)
cat.fit(X_train, y_train)
y_pred_train = cat.predict(X_train)
y_pred_test = cat.predict(X_test)

cat_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
cat_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['cat_train'] = cat_train
df['cat_test'] = cat_test


print(confusion_matrix(y_test,y_pred_test))

[[3156  236]
 [  38  267]]


In [28]:
svc_clf = SVC() 
svc_clf.fit(X_train,y_train)
y_pred_train = svc_clf.predict(X_train)
y_pred_test = svc_clf.predict(X_test)

svc_clf_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
svc_clf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['svm_train'] = svc_clf_train
df['svm_test'] = svc_clf_test


print(confusion_matrix(y_test,y_pred_test))

[[3070  322]
 [  39  266]]


In [29]:
clf = TabNetClassifier()

# 모델 훈련
clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
    eval_name=['train', 'test'],
    eval_metric=['accuracy'],
    max_epochs=10,
    patience=10,
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 0.43494 | train_accuracy: 0.81355 | test_accuracy: 0.92399 |  0:00:02s
epoch 1  | loss: 0.27352 | train_accuracy: 0.88854 | test_accuracy: 0.89343 |  0:00:04s
epoch 2  | loss: 0.24916 | train_accuracy: 0.9012  | test_accuracy: 0.89532 |  0:00:07s
epoch 3  | loss: 0.24506 | train_accuracy: 0.91051 | test_accuracy: 0.88126 |  0:00:09s
epoch 4  | loss: 0.23473 | train_accuracy: 0.90779 | test_accuracy: 0.89803 |  0:00:11s
epoch 5  | loss: 0.23566 | train_accuracy: 0.90969 | test_accuracy: 0.8937  |  0:00:14s
epoch 6  | loss: 0.23284 | train_accuracy: 0.91355 | test_accuracy: 0.88098 |  0:00:16s
epoch 7  | loss: 0.22742 | train_accuracy: 0.91539 | test_accuracy: 0.88342 |  0:00:19s
epoch 8  | loss: 0.2182  | train_accuracy: 0.91906 | test_accuracy: 0.88802 |  0:00:21s
epoch 9  | loss: 0.22022 | train_accuracy: 0.91596 | test_accuracy: 0.89938 |  0:00:23s
Stop training because you reached max_epochs = 10 with best_epoch = 0 and best_test_accuracy = 0.92399


In [30]:
y_pred_train = clf.predict(X_train.values)
y_pred_test = clf.predict(X_test.values)

tab_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
tab_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['tabnet_train'] = tab_train
df['tabnet_test'] = tab_test


print(confusion_matrix(y_test,y_pred_test))

[[3209  183]
 [  98  207]]


In [31]:
df.index = ['accuracy','precision','recall','f1-score','roc-auc']
df

,logit_train,logit_test,rf_train,rf_test,xgb_train,xgb_test,lgb_train,lgb_test,cat_train,cat_test,svm_train,svm_test,tabnet_train,tabnet_test
accuracy,0.884167,0.908034,0.999937,0.929402,0.996263,0.926968,0.979607,0.922640,0.983154,0.925886,0.874478,0.902353,0.813553,0.923992
precision,0.903555,0.468694,0.999874,0.545833,0.993094,0.536383,0.964736,0.518664,0.970816,0.530815,0.897573,0.452381,0.922750,0.530769
recall,0.860735,0.859016,1.000000,0.859016,0.999495,0.845902,0.995703,0.865574,0.996335,0.875410,0.846076,0.872131,0.685328,0.678689
f1-score,0.881626,0.606481,0.999937,0.667516,0.996284,0.656489,0.979975,0.648649,0.983410,0.660891,0.871064,0.595745,0.786512,0.595683
roc-auc,0.884221,0.885729,0.999937,0.897374,0.996256,0.890079,0.979571,0.896672,0.983124,0.902917,0.874542,0.888601,0.813846,0.812369


In [32]:
X_train

,순이익증가율,종업원수증가율,총자본투자효율,부가가치율,매출액순이익률,자기자본정상영업이익률,CASH FLOW 대 매출액비율,CASH FLOW 대 차입금비율
0,0.000000,-8.460000,7.840000,25.180000,9.310000,3.630000,9.610000,4560.740000
1,0.000000,-2.080000,6.100000,12.340000,4.450000,3.140000,22.260000,117.640000
2,-17.587225,1.290000,37.160000,66.510000,4.870000,15.430000,39.365800,57.780000
3,1593.861469,-20.670000,57.950000,32.430000,4.890000,30.200000,2.660000,12.430000
4,-12.414234,2.700000,19.550000,49.550000,17.370000,10.670000,22.710000,4835.120000
...,...,...,...,...,...,...,...,...
15785,-49.621316,-0.882514,8.330823,13.069594,-17.790856,-12.954272,-3.549820,-5.838050
15786,-94.079450,12.052610,-0.003868,0.431405,-21.378715,-27.917167,-21.671934,-31.334646
15787,58.768439,-12.266795,19.052934,49.888405,-9.865584,-7.043025,22.453030,17.438913
15788,81.380642,-7.525738,11.711767,19.373171,-0.553820,3.148905,-0.911838,-1.482571
